# 本セクションの目次
1. Avroフォーマット
2. 前方互換と後方互換と完全互換
3. メッセージキューとAvroを連携してみよう
4. Avroファイルの読み書き
5. Avroで前方互換をやってみよう

In [1]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init("/home/pyspark/spark")

In [2]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .config("spark.jars.packages", "org.apache.spark:spark-streaming_2.13:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-avro_2.12:3.2.0") \
    .enableHiveSupport() \
    .getOrCreate()

# パッケージを複数渡したい時は「,」で繋いで渡します。
# Sparkのバージョンにしっかりと合わせます(今回はSparkのバージョンが3.2を使っています。)。

:: loading settings :: url = jar:file:/home/pyspark/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pyspark/.ivy2/cache
The jars for the packages stored in: /home/pyspark/.ivy2/jars
org.apache.spark#spark-streaming_2.13 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a28822ed-5037-4a99-814c-64304dc0d58b;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in ce

# ビッグデータの世界のDDL

ビッグデータの世界でのDDLはRDSと同じ様にDDL文を実行することが可能です。  
今回は以下のDDLについてみていきましょう  

- Create Database 文
- CREATE EXTERNAL TABLE文
- CREATE VIEW
- ADD PARTITION(MSCK REPAIR)

## CREATE DATABASE文
データベースの作成を行います。
こちらはRDSなどのCreate Database　と同じ方法で作成が可能です。

In [3]:
spark.sql("create database if not exists super_crush_course")

22/01/05 06:34:53 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
22/01/05 06:34:53 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
Wed Jan 05 06:34:54 UTC 2022 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.
Wed Jan 05 06:34:54 UTC 2022 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not us

DataFrame[]

In [7]:
# データベースの一覧を見てみましょう
spark.sql("show databases").show(truncate=False)

+----------------------------+
|namespace                   |
+----------------------------+
|data_management_crush_course|
|default                     |
|metadata_tmp                |
|super_crush_course          |
+----------------------------+



## CREATE EXTERNAL TABLE文
テーブル定義の構成要素をみていきましょう

```
CREATE EXTERNAL TABLE IF NOT EXISTS super_crush_course.csv_table ( id INT, date STRING)
PARTITIONED BY (dt INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LOCATION '/home/pyspark/super_crush_course.db/csv_table/dataset/parquet/';

#S3などであれば、以下のように設定を変えることも可能です。
LOCATION 's3://data.platform/super_crush_course.db/raw_zone/sampletable/';

```

ビッグデータの世界では、実データとデータベース定義/テーブル定義(メタデータ)は明確に分離されています。  
今回のコースだと実データはローカル端末のSSD(HDD)でテーブル定義はMysqlに登録されています。  
明確に分離されているからこそ、場所を指し示す宣言であるLocationが必要になってきます  

ロケーションは「super_crush_course.db」とDB名.db/TABLE名とすることが通例です。  
Externalは外部のという意味で、オンプレ環境の場合はつけないことが多かったが、クラウド環境ではつけるのが必須となっている設定。

メタデータについてさらに詳しく知りたい方は、以下の記事を参照してみてください  
「【PythonとSparkで始めるデータマネジメント入門】 ビッグデータレイクのための統合メタデータ管理入門」

上記の例は、CSV形式のテーブルです。  
それ以外にもParquet形式、Avro形式でテーブルを作成することができます。

```
# パーティションつきのテーブル
CREATE TABLE IF NOT EXISTS super_crush_course.parquet_table 
(code String, gengo String,wareki String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
PARTITIONED BY (kenmei String)
STORED AS PARQUET
TBLPROPERTIES ("parquet.compression"="SNAPPY");
LOCATION '/home/pyspark/super_crush_course.db/parquet_table';

```

圧縮形式と保存するファイルフォーマットを指定してテーブルの作成を行なっていきます。  
ちなみにテーブル定義のカラムは、今回読み込んだCSV/JSONのスキーマになります。  
Partitionとはデータの区切りのことで、kenmeiを区切りとしてデータを保存しています(次のレクチャーにて)。　 

```
# パーティションなしのテーブルの場合
CREATE TABLE IF NOT EXISTS super_crush_course.parquet_table_with_no_partition
(code String, gengo String,kenmei, Stringwareki String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
STORED AS PARQUET
TBLPROPERTIES ('parquet.compression'='SNAPPY')
LOCATION '/home/pyspark/super_crush_course.db/parquet_table_with_no_partition'

```

In [11]:
## テーブル作成
# テーブル作成も同様に、spark.sqlを使ってテーブルを作成していきます。
# ロケーションはセクション2で出力したディレクトリになります。

# Parquetテーブルの作成(パーティションあり)
spark.sql("""

CREATE TABLE IF NOT EXISTS super_crush_course.parquet_table_with_partition 
(code String, gengo String,wareki String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
PARTITIONED BY (kenmei String)
STORED AS PARQUET
TBLPROPERTIES ('parquet.compression'='SNAPPY')
LOCATION '/home/pyspark/super_crush_course.db/parquet_table_with_partition'

""")

# Paruqetテーブルの作成(パーティションなし)
spark.sql("""

CREATE TABLE IF NOT EXISTS super_crush_course.parquet_table_with_no_partition 
(code String, gengo String,wareki String,kenmei String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
STORED AS PARQUET
TBLPROPERTIES ('parquet.compression'='SNAPPY')
LOCATION '/home/pyspark/super_crush_course.db/parquet_table_with_no_partition'

""")

DataFrame[]

In [12]:
# AVROテーブルの作成(パーティションあり)
spark.sql("""

CREATE TABLE IF NOT EXISTS super_crush_course.avro_table_with_partition 
(code String, gengo String,wareki String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
PARTITIONED BY (kenmei String)
STORED AS AVRO
TBLPROPERTIES ('parquet.compression'='SNAPPY')
LOCATION '/home/pyspark/super_crush_course.db/parquet_table_with_partition'

""")

# AVROテーブルの作成(パーティションなし)
spark.sql("""

CREATE TABLE IF NOT EXISTS super_crush_course.avro_table_with_no_partition 
(code String, gengo String,wareki String,kenmei String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
STORED AS AVRO
TBLPROPERTIES ('parquet.compression'='SNAPPY')
LOCATION '/home/pyspark/super_crush_course.db/avro_table_with_no_partition'

""")

DataFrame[]

In [14]:
# 作成したテーブルを見てみましょう
spark.sql("show tables in super_crush_course").show(truncate=False)

+------------------+-------------------------------+-----------+
|namespace         |tableName                      |isTemporary|
+------------------+-------------------------------+-----------+
|super_crush_course|avro_table_with_no_partition   |false      |
|super_crush_course|avro_table_with_partition      |false      |
|super_crush_course|parquet_table_with_no_partition|false      |
|super_crush_course|parquet_table_with_partition   |false      |
+------------------+-------------------------------+-----------+



# ADD PRTITION
パーティションを認識するためにコマンドを発行する必要があります。




In [18]:
# パーティションも管理されている
spark.sql("show partitions super_crush_course.parquet_table_with_partition").show(n=2)
# 本来件名があってほしいが。。。

+---------+
|partition|
+---------+
+---------+



In [19]:
# パーティションがテーブルは。。？クエリしてみる
spark.sql("select * from super_crush_course.parquet_table_with_no_partition").show(n=2)

+-----------------------------------+-----+------+------+-------+-------+------+----------+------------+
|                               code|gengo|wareki|kenmei|seireki|    chu| sokei|jinko_male|jinko_femail|
+-----------------------------------+-----+------+------+-------+-------+------+----------+------------+
|2)　長野県西筑摩群山口村と岐阜県...| null|  null|  null|   null|   null|  null|      null|        null|
|                                 38| 昭和|    60|  1985|   null|1529983|728506|      null|      愛媛県|
+-----------------------------------+-----+------+------+-------+-------+------+----------+------------+
only showing top 2 rows



In [20]:
# パーティションがあってadd paritionをしていないテーブルは。。？クエリしてみる
spark.sql("select * from super_crush_course.parquet_table_with_partition").show(n=2)
# データを見ることができない

+----+-----+------+-------+---+-----+----------+------------+------+
|code|gengo|wareki|seireki|chu|sokei|jinko_male|jinko_femail|kenmei|
+----+-----+------+-------+---+-----+----------+------------+------+
+----+-----+------+-------+---+-----+----------+------------+------+



In [41]:
# パーティションを追加してみる
# パーティションの追加には２種類存在しています
# add partition
# msck repair table名
spark.sql("alter table super_crush_course.parquet_table_with_partition add if not exists  partition (kenmei='東京都')")

DataFrame[]

In [47]:
# パーティションも管理されている
spark.sql("show partitions super_crush_course.parquet_table_with_partition").show(n=20)

+-----------------------------+
|                    partition|
+-----------------------------+
|         kenmei=__HIVE_DEF...|
|                kenmei=三重県|
|                kenmei=京都府|
|          kenmei=人口集中地区|
|kenmei=人口集中地区以外の地区|
|                kenmei=佐賀県|
|                  kenmei=全国|
|                kenmei=兵庫県|
|                kenmei=北海道|
|                kenmei=千葉県|
|              kenmei=和歌山県|
|                kenmei=埼玉県|
|                kenmei=大分県|
|                kenmei=大阪府|
|                kenmei=奈良県|
|                kenmei=宮城県|
|                kenmei=宮崎県|
|                kenmei=富山県|
|                kenmei=山口県|
|                kenmei=山形県|
+-----------------------------+
only showing top 20 rows



In [48]:
# 再度検索を行ってみる
# パーティションがあってadd paritionをしていないテーブルは。。？クエリしてみる
spark.sql("select * from super_crush_course.parquet_table_with_partition").show()
# 追加した東京都のデータだけ見える

+-----------------------------------+-----+------+-------+----+-------+----------+------------+------+
|                               code|gengo|wareki|seireki| chu|  sokei|jinko_male|jinko_femail|kenmei|
+-----------------------------------+-----+------+-------+----+-------+----------+------------+------+
|2)　長野県西筑摩群山口村と岐阜県...| null|  null|   null|null|   null|      null|        null|  null|
|                                 38| 昭和|    60|   1985|null|1529983|    728506|        null|愛媛県|
|                                 38| 昭和|    25|   1950|null|1521878|    742092|        null|愛媛県|
|                                 38| 平成|     2|   1990|null|1515025|    716940|        null|愛媛県|
|                                 38| 昭和|    30|   1955|null|1540628|    749342|        null|愛媛県|
|                                 38| 昭和|    55|   1980|null|1506637|    718517|        null|愛媛県|
|                                 38| 昭和|    40|   1965|null|1446384|    688063|        null|愛媛県|
|              

In [45]:
# 一個づつAdd partitionするのは面倒なのでmsckを使う(ただし時間がかかる場合が多いので、日々の処理であればadd partitionを選択する方がいい)
spark.sql("msck repair table super_crush_course.parquet_table_with_partition")
spark.sql("msck repair table super_crush_course.avro_table_with_partition")

DataFrame[]

# Create View
ビューを作成します。
ビューとは、仮想的なテーブルのことでデータを生成しなくてもテーブルを生成することが可能です。

言葉で伝えるより実際に見た方がいいと思うので、早速作ってみましょう。　　

手っ取り早くクエリを簡単にしたい場合に有効です。

In [56]:
# create view

spark.sql("""

create view parquet_view (gengo)
as 
select gengo from 
super_crush_course.parquet_table_with_partition

""")

DataFrame[]

In [57]:
spark.sql("select * from parquet_view").show()

+-----+
|gengo|
+-----+
| null|
| 昭和|
| 昭和|
| 平成|
| 昭和|
| 昭和|
| 昭和|
| 昭和|
| 昭和|
| 大正|
| 昭和|
| 昭和|
| 平成|
| 平成|
| 昭和|
| 昭和|
| 昭和|
| 平成|
| 平成|
| 昭和|
+-----+
only showing top 20 rows



# テンポラリテーブル
テンポラリーテーブルとはデータフレームから一時的にテーブルを作成することで、SparkSessionごとに生成が可能です。

特にスキーマオンリードで読み込んだdataframeを一時的にテーブルにすることで、SQLでの操作を可能にすることができます。

一連の流れを見てみましょう

In [58]:
# テンポラリーテーブルの作成
json_df=spark.read.json("./dataset/jinko.json")
json_df.createOrReplaceTempView("json_tmp")

In [59]:
#　json_tempの検索
spark.sql("select * from json_tmp where kenmei='東京都'").show()

+----+----+-----+------------+----------+------+-------+--------+------+
| chu|code|gengo|jinko_female|jinko_male|kenmei|seireki|   sokei|wareki|
+----+----+-----+------------+----------+------+-------+--------+------+
|null|  13| 大正|     1746439|   1952989|東京都|   1920| 3699428|     9|
|null|  13| 大正|     2097535|   2387609|東京都|   1925| 4485144|    14|
|null|  13| 昭和|     2553355|   2855323|東京都|   1930| 5408678|     5|
|null|  13| 昭和|     3044223|   3325696|東京都|   1935| 6369919|    10|
|null|  13| 昭和|     3559096|   3795875|東京都|   1940| 7354971|    15|
|  1)|  13| 昭和|     1700139|   1788145|東京都|   1945| 3488284|    20|
|null|  13| 昭和|     3108111|   3169389|東京都|   1950| 6277500|    25|
|null|  13| 昭和|     3921261|   4115823|東京都|   1955| 8037084|    30|
|  2)|  13| 昭和|     4686779|   4997023|東京都|   1960| 9683802|    35|
|null|  13| 昭和|     5304661|   5564583|東京都|   1965|10869244|    40|
|null|  13| 昭和|     5607062|   5801009|東京都|   1970|11408071|    45|
|null|  13| 昭和|     5760181|   59

# ビッグデータ世界のDMLとは

ビッグデータの世界のSQLは基本的にSQL’ライク’です。  
というのもRDSのSQLを前提にしてライクと言っているだけなので、ビッグデータ世界を中心としたらSQLそのものです。  
RDSでのSQLに慣れている人は、ビッグデータの世界のSQLは難なくこなすことができると思います。  

- SELECT
- CTAS
- SELECT INSERT
- INSERT?
- UPDATE?
- DELETE?

今回は4つのDMLを確認してみます。

In [62]:
# SELECT
spark.sql("select * from super_crush_course.parquet_table_with_no_partition").show()

## パーティションつきのテーブルを検索してみる
spark.sql("select * from super_crush_course.parquet_table_with_partition where kenmei ='東京都'").show()

# パーティションなしの場合は、すべてのデータを走査してから絞り込みます
# パーティションありの場合は、特定のパーティション配下のディレクトリのみをチェックします

# 大体のRDSのSQLでできることは実行可能です。

+----+-----+------+----------------------+-------+----+--------+----------+------------+
|code|gengo|wareki|                kenmei|seireki| chu|   sokei|jinko_male|jinko_femail|
+----+-----+------+----------------------+-------+----+--------+----------+------------+
|  25| 平成|    22|                滋賀県|   2010|null| 1410777|    696769|        null|
|  27| 昭和|    10|                大阪府|   1935|null| 4297174|   2241666|        null|
|  28| 昭和|    45|                兵庫県|   1970|null| 4667928|   2299961|        null|
|  32| 平成|     7|                島根県|   1995|null|  771441|    368789|        null|
|  27| 昭和|    60|                大阪府|   1985|null| 8668095|   4286445|        null|
|  38| 昭和|    60|                愛媛県|   1985|null| 1529983|    728506|        null|
|  13| 昭和|    50|                東京都|   1975|null|11673554|   5913373|        null|
|  12| 昭和|    35|                千葉県|   1960|  2)| 2306010|   1128734|        null|
|  10| 昭和|    50|                群馬県|   1975|null| 1756480|  

Wed Jan 05 08:23:16 UTC 2022 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.
Wed Jan 05 08:23:16 UTC 2022 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for s

# CTAS
Create Table As Selectの略です。

簡単にいうと、Selectの返却結果からテーブルを作成することが可能です。

In [ ]:
# CTASの実演

# SELECT INSERT
SELECT INSERTの場合は、ADD PARTITIONは不要です

In [ ]:
# SELECT INSERTの実演

# INSERT/UPDATE/DELETE?
ビッグデータの世界では原則としてACIDをサポートしていません。  
そのため、UPDATEやDELETがサポートされていないことが多いです。

INSERTは単体で利用することはできますが、あまり出番がなく前述で紹介したSELECT INSERTでの出番が大半です。

# 分析関数の練習をしよう
ここからは、分析関数の練習をしてみましょう。  

- agg
- window(over)
- ピボットテーブル
- lag関数

データフレームでの操作とSQLでの操作を対比させながら実行していきます。

# ピボットテーブル

# LAG関数

# データフレーム限定 RDDによる一行づつの操作

In [ ]:
spark.stop()